In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession
session = SnorkelSession()

import os
from snorkel.parser import XMLMultiDocPreprocessor

# The following line is for testing only. Feel free to ignore it.
file_path = 'articles/training.xml'
train_preprocessor = XMLMultiDocPreprocessor(
    path=file_path,
    doc='.//article',
    text='.//front/article-meta/abstract/p/text()',
    id=  './/front/article-meta/article-id/text()'
)

file_path = 'articles/development.xml'
dev_preprocessor = XMLMultiDocPreprocessor(
    path=file_path,    
    doc='.//document',    
    text='.//passage/text/text()',    
    id='.//id/text()'
)


In [ ]:
from snorkel.parser import CorpusParser

corpus_parser = CorpusParser()
corpus_parser.apply(list(train_preprocessor)) #parallelism can be run with a Postgres DBMS, but not SQLite
corpus_parser.apply(list(dev_preprocessor), clear=False)

In [ ]:
from six.moves.cPickle import load

with open('articles/doc_ids.pkl', 'rb') as f:
    train_ids, dev_ids = load(f)
train_ids, dev_ids = set(train_ids), set(dev_ids)
print len(train_ids)
print len(dev_ids)
train_sents, dev_sents = set(), set()
docs = session.query(Document).order_by(Document.name).all()
print docs
print len(docs)
for i, doc in enumerate(docs):
    for s in doc.sentences:
        if doc.name in train_ids:
            train_sents.add(s)
        elif doc.name in dev_ids:
            dev_sents.add(s)
        else:
            raise Exception('ID <{0}> not found in any id set'.format(doc.name))

In [ ]:
from snorkel.models import Candidate, candidate_subclass

BiomarkerDrug = candidate_subclass('BiomarkerDrug', ['biomarker', 'drug'])

from snorkel.models import candidate_subclass
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import PersonMatcher
import matchers
from snorkel.models import Document
from snorkel.viewer import SentenceNgramViewer
import os

biomarker_ngrams = Ngrams(n_max=1)
drug_ngrams = Ngrams(n_max=5)

# Create our two Matchers
bMatcher = matchers.getBiomarkerMatcher()
dMatcher = matchers.getDrugMatcher()
    
# Building the CandidateExtractor 
candidate_extractor = CandidateExtractor(BiomarkerDrug, [biomarker_ngrams, drug_ngrams], [bMatcher, dMatcher])


In [ ]:
for k, sents in enumerate([train_sents, dev_sents]):
    candidate_extractor.apply(sents, split=k)
    print("Number of candidates:", session.query(BiomarkerDrug).filter(BiomarkerDrug.split == k).count())

In [ ]:
keyWords = ['basis', 'target', 'treat', 'treatment', 'inhibit', 'inhibition', 'inhibitor', ]
negationWords = ["not", "nor", "neither"]

def presenceOfNot(m):
    for word in negationWords:
        if (word in m.post_window1('lemmas', 20)) and (word in m.pre_window2('lemmas', 20)):
            return True
    return False

# 1
def LF_distance(m):
    # if 'neuroendocrine' in m.lemmas:
    #     print m.lemmas 
    # print m.dep_labels
    distance = abs(m.e2_idxs[0] - m.e1_idxs[0])
    count = 0
    for lemma in m.lemmas:
        if lemma == ',':
            count += 1
    if count > 1 and ',' in m.pre_window1('lemmas', 1):
        print m
        return 0
    if distance == 0:
        return -1
    if distance < 8:
        # print "RETURNING ONE"
        return 0
    else:
        return -1

def LF_keyword(m):
    for word in keyWords:
        if ((word in m.post_window1('lemmas', 20)) and (word in m.pre_window2('lemmas', 20))) or ((word in m.post_window1('lemmas', 20)) and (word in m.pre_window2('lemmas', 20))):
            if presenceOfNot(m):
                return -1
            else:
                return 1
    return 0

def LF_usedToTreat(m):
     if ('used' in m.pre_window1('lemmas'), 20) and ('to' in m.pre_window1('lemmas', 20)) and ('treat' in m.pre_window1('lemmas', 20)):
         return 1
     else:
         return 0
     
def LF_usedToInhibit(m):
     if ('used' in m.pre_window1('lemmas'), 20) and ('to' in m.pre_window1('lemmas', 20)) and ('inhibit' in m.pre_window1('lemmas', 20)):
         return 1
     else:
         return 0
     
def LF_inhibit(m):
    if ('inhibit' in m.pre_window1('lemmas', 20) and 'inhibit' in m.pre_window2('lemms', 20)):
        return 1
    else:
        return 0
    
def LF_target(m):
    if ('target' in m.pre_window1('lemmas', 20) and 'target' in m.pre_window2('lemms', 20)):
        return 1
    else:
        return 0

def LF_inhibitionOfWith(m):
    if ('inhibition' in m.pre_window1('lemmas', 20) and 'of' in m.pre_window1('lemmas', 20) and 'with' in m.post_window1('lemmas', 20)):
        return 1
    else:
        return 0
    
def LF_isATreatmentOf(m):
    if ('is' in m.pre_window1('lemmas', 20) and 'a' in m.pre_window1('lemmas', 20) and 'treatment' in m.pre_window1('lemmas', 20) and 'of' in m.pre_window1('lemmas', 20)):
        return 1
    else:
        return 0
    
def LF_antibody(m):
    if ('antibody' in m.post_window2('lemmas', 20)):
        return 1
    else:
        return 0
def LF_hadisbad(m):
    if(m.mention1 == "had" or m.mention2 == "had"):
        return -1
def LF_duration(m):
    if(m.mention1 == "duration" or m.mention2 == "duration"):
        return -1
def LF_isaBiomarker(m):
    post_window1_lemmas = m.post_window1('lemmas',20)
    pre_window2_lemmas = m.pre_window2('lemmas',20)
    if ('biomarker' in post_window1_lemmas and 'biomarker' in pre_window2_lemmas) or ('marker' in post_window1_lemmas and 'marker' in pre_window2_lemmas) or ('indicator' in post_window1_lemmas and 'indicator' in pre_window2_lemmas):
        marker_idx_post_window1 = -1
        markers = ['biomarker','marker','indicator']
        for marker in markers:
            try:
                # print post_window1_lemmas
                findMarker = post_window1_lemmas.index(marker)
                if not findMarker == -1:
                    marker_idx_post_window1 = findMarker
                    print marker
            except:
                pass
        if 'cop' in m.post_window1('dep_labels',20):
            try:
                cop_idx_post_window1 = m.post_window1('dep_labels',20).index('cop')
            except:
                pass
            
            print "MarkerIdx:"
            print marker_idx_post_window1
            print "ROOTIdx:"
            try:
                print  m.post_window1('dep_labels',marker_idx_post_window1)
                print  m.post_window1('dep_labels',marker_idx_post_window1).index('ROOT')
            except:
                pass
            print '\n'
            
            return 1 if ('nsubj' in m.mention1(attribute='dep_labels')) and (marker_idx_post_window1-cop_idx_post_window1 < 4)  else 0
    return 0

In [ ]:
LFs = [LF_distance, LF_keyword, LF_usedToTreat, LF_inhibit, LF_inhibitionOFWith, LF_isaBiomarker]

In [ ]:
from snorkel.annotations import LabelAnnotator
labeler = LabelAnnotator(lfs=LFs)

In [ ]:
%time L_train = labeler.apply(split=0)
L_train

In [ ]:
L_train.lf_stats(session)

In [ ]:
from snorkel.learning.structure import DependencySelector
ds = DependencySelector()
deps = ds.select(L_train, threshold=0.1)
len(deps)

In [ ]:
deps

In [ ]:
deps = set()

In [ ]:
from snorkel.learning import GenerativeModel

gen_model = GenerativeModel(lf_propensity=True)
gen_model.train(
    L_train, deps=deps, decay=0.95, step_size=0.1/L_train.shape[0], reg_param=0.0
)

In [ ]:
train_marginals = gen_model.marginals(L_train)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(train_marginals, bins=20)
plt.show()

In [ ]:
gen_model.learned_lf_stats()

In [ ]:
from snorkel.annotations import save_marginals
save_marginals(session, L_train, train_marginals)

In [ ]:
print L_gold_dev
L_dev = labeler.apply_existing(split=1)
print L_gold_dev

In [ ]:
from load_external_annotations_new import load_external_labels
load_external_labels(session, BiomarkerDrug, annotator_name='gold')

In [ ]:
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev

In [ ]:
_ = gen_model.score(session, L_dev, L_gold_dev)

In [ ]:
L_dev.lf_stats(session, L_gold_dev, gen_model.learned_lf_stats()['Accuracy'])

In [ ]:
import numpy as np

# Load dev labels and convert to [0, 1] range
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
dev_labels = (np.ravel(L_gold_dev.todense()) + 1) / 2

In [ ]:
from snorkel.annotations import FeatureAnnotator
featurizer = FeatureAnnotator()

%time F_train = featurizer.apply(split=0)
F_train

In [ ]:
%%time
F_dev  = featurizer.apply_existing(split=1)
F_test = featurizer.apply_existing(split=2)

In [ ]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, F_train, split=0)

In [ ]:
from snorkel.learning import SparseLogisticRegression
disc_model = SparseLogisticRegression()

In [ ]:
from snorkel.learning.utils import MentionScorer
from snorkel.learning import RandomSearch, ListParameter, RangeParameter

# Searching over learning rate
rate_param = RangeParameter('lr', 1e-6, 1e-2, step=1, log_base=10)
l1_param  = RangeParameter('l1_penalty', 1e-6, 1e-2, step=1, log_base=10)
l2_param  = RangeParameter('l2_penalty', 1e-6, 1e-2, step=1, log_base=10)

searcher = RandomSearch(session, disc_model, F_train, train_marginals, [rate_param, l1_param, l2_param], n=20)

In [ ]:
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev

import numpy as np
np.random.seed(1701)
searcher.fit(F_dev, L_gold_dev, n_epochs=50, rebalance=0.5, print_freq=25)

### Scoring on the test set

Finally, we'll evaluate our performance on the blind test set of 500 documents. We'll load labels similar to how we did for the development set, and use the `score` function of our extraction model to see how we did.

from load_external_annotations import load_external_labels
load_external_labels(session, BiomarkerDrug, split=2, annotator='gold')

from snorkel.annotations import load_gold_labels
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_test

_, _, _, _ = disc_model.score(session, F_test, L_gold_test)

In [ ]:
from snorkel.contrib.rnn import reRNN

train_kwargs = {
    'lr':         0.01,
    'dim':        100,
    'n_epochs':   50,
    'dropout':    0.5,
    'rebalance':  0.25,
    'print_freq': 5
}

lstm = reRNN(seed=1701, n_threads=None)
lstm.train(train, train_marginals, dev_candidates=dev, dev_labels=dev_labels, **train_kwargs)